# 01. 初始資料探索

這個 notebook 進行初步的資料探索，對應第1週的任務：
- 載入 Kaggle 房價資料集
- 初步資料品質評估
- 基本統計分析
- 識別資料問題

## 1. 環境設定

In [1]:
# 設定路徑，讓我們能夠導入 src 模組
import sys
from pathlib import Path

# 將專案根目錄加入 Python 路徑
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 導入必要的套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rich import print
import warnings

# 設定顯示選項
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# 設定視覺化風格
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# 導入我們的模組
from src.data.ingestion import DataIngestion
from src.utils.logger import get_logger

logger = get_logger('01_initial_exploration')
logger.info('開始初始資料探索')

[06/09/25 16:36:56] INFO     開始初始資料探索                                                          ]8;id=982979;file://D:\house_price_prediction\src\utils\logger.py\logger.py]8;;\:]8;id=866520;file://D:\house_price_prediction\src\utils\logger.py#97\97]8;;\

## 2. 載入資料

In [2]:
# 建立資料載入器
data_loader = DataIngestion()

# 載入資料（如果資料不存在會自動下載）
try:
    train_df, test_df = data_loader.load_data()
    print(f"[green]✓ 資料載入成功！[/green]")
    print(f"訓練資料: {train_df.shape}")
    print(f"測試資料: {test_df.shape}")
except Exception as e:
    print(f"[red]✗ 資料載入失敗: {str(e)}[/red]")
    print("請確認：")
    print("1. 是否已設定 Kaggle API (~/.kaggle/kaggle.json)")
    print("2. 或手動下載資料到 data/raw/ 目錄")

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'NoneType'

## 3. 資料基本資訊

In [ ]:
# 顯示前幾筆資料
print("[bold]訓練資料前5筆：[/bold]")
train_df.head()

In [ ]:
# 資料基本資訊
print("[bold]資料型態統計：[/bold]")
print(train_df.dtypes.value_counts())
print("\n[bold]數值型欄位：[/bold]")
numeric_columns = train_df.select_dtypes(include=[np.number]).columns.tolist()
print(f"共 {len(numeric_columns)} 個數值型欄位")
print("\n[bold]類別型欄位：[/bold]")
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
print(f"共 {len(categorical_columns)} 個類別型欄位")

## 4. 目標變數分析

In [ ]:
# 目標變數（房價）的分佈
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 直方圖
axes[0].hist(train_df['SalePrice'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('房價分佈')
axes[0].set_xlabel('房價 (USD)')
axes[0].set_ylabel('頻率')

# Q-Q plot
from scipy import stats
stats.probplot(train_df['SalePrice'], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot')

# 對數轉換後的分佈
log_price = np.log1p(train_df['SalePrice'])
axes[2].hist(log_price, bins=50, edgecolor='black', alpha=0.7)
axes[2].set_title('對數房價分佈')
axes[2].set_xlabel('log(房價)')
axes[2].set_ylabel('頻率')

plt.tight_layout()
plt.show()

# 統計描述
print("[bold]房價統計摘要：[/bold]")
print(train_df['SalePrice'].describe())
print(f"\n偏度 (Skewness): {train_df['SalePrice'].skew():.3f}")
print(f"峰度 (Kurtosis): {train_df['SalePrice'].kurtosis():.3f}")

## 5. 缺失值分析

In [ ]:
# 計算缺失值
missing_train = train_df.isnull().sum()
missing_train = missing_train[missing_train > 0].sort_values(ascending=False)
missing_percent = (missing_train / len(train_df)) * 100

# 建立缺失值統計表
missing_df = pd.DataFrame({
    '缺失數量': missing_train,
    '缺失比例': missing_percent
})

print("[bold]訓練資料缺失值統計（前20個）：[/bold]")
print(missing_df.head(20))

# 視覺化缺失值
plt.figure(figsize=(12, 8))
missing_df.head(20).plot(kind='barh')
plt.title('缺失值統計（前20個特徵）')
plt.xlabel('數量 / 比例')
plt.tight_layout()
plt.show()

In [ ]:
# 缺失值模式分析
# 檢查高缺失率特徵的含義
high_missing_features = missing_df[missing_df['缺失比例'] > 80].index.tolist()

print("[bold]高缺失率特徵分析：[/bold]")
for feature in high_missing_features:
    print(f"\n{feature} (缺失 {missing_df.loc[feature, '缺失比例']:.1f}%)")
    if feature == 'PoolQC':
        print("  → 游泳池品質：大部分房屋可能沒有游泳池")
    elif feature == 'MiscFeature':
        print("  → 其他特徵：大部分房屋可能沒有特殊設施")
    elif feature == 'Alley':
        print("  → 巷道類型：大部分房屋可能沒有巷道通行")
    elif feature == 'Fence':
        print("  → 圍欄品質：大部分房屋可能沒有圍欄")

## 6. 數值特徵初步分析

In [ ]:
# 重要數值特徵的統計
important_numeric = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'GarageArea', 
                    'YearBuilt', 'OverallQual', 'OverallCond']

print("[bold]重要數值特徵統計：[/bold]")
train_df[important_numeric].describe()

In [ ]:
# 數值特徵與房價的相關性
# 選擇與房價相關性較高的特徵
correlations = train_df[numeric_columns].corr()['SalePrice'].sort_values(ascending=False)
top_corr_features = correlations.head(15)

# 視覺化
plt.figure(figsize=(10, 8))
top_corr_features.plot(kind='barh')
plt.title('與房價相關性最高的15個特徵')
plt.xlabel('相關係數')
plt.tight_layout()
plt.show()

print("[bold]前10個最相關特徵：[/bold]")
for feature, corr in top_corr_features.head(10).items():
    if feature != 'SalePrice':
        print(f"{feature}: {corr:.3f}")

## 7. 初步異常值檢測

In [ ]:
# 檢查極端值
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# GrLivArea vs SalePrice
axes[0, 0].scatter(train_df['GrLivArea'], train_df['SalePrice'], alpha=0.5)
axes[0, 0].set_xlabel('地上居住面積')
axes[0, 0].set_ylabel('房價')
axes[0, 0].set_title('居住面積 vs 房價')

# 標記可能的異常值
outliers = train_df[train_df['GrLivArea'] > 4000]
axes[0, 0].scatter(outliers['GrLivArea'], outliers['SalePrice'], 
                   color='red', s=100, marker='x')

# TotalBsmtSF vs SalePrice
axes[0, 1].scatter(train_df['TotalBsmtSF'], train_df['SalePrice'], alpha=0.5)
axes[0, 1].set_xlabel('地下室總面積')
axes[0, 1].set_ylabel('房價')
axes[0, 1].set_title('地下室面積 vs 房價')

# YearBuilt distribution
axes[1, 0].hist(train_df['YearBuilt'], bins=50, edgecolor='black')
axes[1, 0].set_xlabel('建造年份')
axes[1, 0].set_ylabel('頻率')
axes[1, 0].set_title('建造年份分佈')

# OverallQual vs SalePrice boxplot
train_df.boxplot(column='SalePrice', by='OverallQual', ax=axes[1, 1])
axes[1, 1].set_xlabel('整體品質評分')
axes[1, 1].set_ylabel('房價')
axes[1, 1].set_title('品質評分 vs 房價')

plt.tight_layout()
plt.show()

# 異常值統計
print("[bold]潛在異常值：[/bold]")
print(f"居住面積 > 4000 sqft: {len(outliers)} 筆")
print(f"房價 > $700,000: {len(train_df[train_df['SalePrice'] > 700000])} 筆")
print(f"建造年份 < 1900: {len(train_df[train_df['YearBuilt'] < 1900])} 筆")

## 8. 儲存初步分析結果

In [ ]:
# 儲存資料基本資訊
data_loader.save_data_info(train_df, test_df)

# 建立初步分析報告
initial_analysis = {
    'data_shape': {
        'train': train_df.shape,
        'test': test_df.shape
    },
    'target_stats': {
        'mean': train_df['SalePrice'].mean(),
        'median': train_df['SalePrice'].median(),
        'std': train_df['SalePrice'].std(),
        'skewness': train_df['SalePrice'].skew(),
        'kurtosis': train_df['SalePrice'].kurtosis()
    },
    'missing_features': missing_df.to_dict(),
    'high_corr_features': top_corr_features.head(10).to_dict(),
    'numeric_features': len(numeric_columns),
    'categorical_features': len(categorical_columns)
}

# 儲存分析結果
import json
with open('reports/initial_analysis.json', 'w') as f:
    json.dump(initial_analysis, f, indent=2, default=str)

print("[green]✓ 初步分析完成！結果已儲存到 reports/ 目錄[/green]")

## 9. 下一步計劃

基於初步分析，我們識別了以下重點：

### 資料品質問題：
1. **缺失值處理**：某些特徵有大量缺失值，需要分析是結構性缺失還是隨機缺失
2. **異常值處理**：發現一些極端值需要進一步驗證
3. **目標變數轉換**：房價呈右偏分佈，需要對數轉換

### 特徵工程機會：
1. **高相關特徵**：OverallQual, GrLivArea, GarageCars 等與房價高度相關
2. **時間特徵**：可以從年份創建房齡等衍生特徵
3. **面積特徵**：可以組合多個面積特徵創建總面積

### 第2週任務預覽：
- 深度 EDA 分析
- 完整的缺失值處理策略
- 異常值深入分析
- 建立基線模型